In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [ ]:
contracts = pd.DataFrame.from_csv("~/contracts.csv")

In [ ]:
contracts.ix[724]

In [ ]:
options = pd.DataFrame.from_csv("~/options.csv")

In [ ]:
options.ix[195035]

In [ ]:
def convert_mat_date(matlab_datenum):
    """
    Matlab to Python date convert helper function
    """
    return datetime.fromordinal(int(matlab_datenum)) + timedelta(days=int(matlab_datenum%1)) - timedelta(days=366)

In [ ]:
from scipy.io import loadmat, savemat
mat = loadmat('/home/ubertrader/Downloads/strategy_210301.mat')

In [ ]:
mat_date = mat['optStr']['seriesDates'][0][0][0]
dates = list(map(convert_mat_date, mat_date))

In [ ]:
is_rollover = mat['optStr']['rolloverDay'][0][0][0]

In [ ]:
is_rollover

In [ ]:
def get_leg_array_ids(mat, leg_index):
    res = np.zeros_like(mat['optStr']['optLegName'][0][0][0])
    
    for i in range(len(res)):
        res[i] = mat['optStr']['optLegName'][0][0][leg_index][i][0][0]
    
    return res    

def get_leg_array_price(mat, leg_index):
    res = mat['optStr']['realLegSeries'][0][0][leg_index]
    return res    

In [ ]:
leg_ids = get_leg_array_ids(mat, 0)
leg_price = get_leg_array_price(mat, 0)

In [ ]:
mat['optStr']['realLegSeries'][0][0][1]

In [ ]:
def leg_contract_series(leg_ids, leg_price, dates, contracts_index):
    if len(leg_ids) != len(dates):
        raise Exception('if len(leg_ids) != len(dates):')
    contracts = []
    
    for i in range(len(leg_ids)):
        con_id = leg_ids[i]
        
        name = contracts_index.ix[con_id]['optionname']
        contracts.append(name)
        
    return pd.DataFrame({'name': contracts, 'price': leg_price}, index=dates)

In [ ]:
leg_series = leg_contract_series(leg_ids, leg_price, dates, options)

In [ ]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')


In [ ]:
print_full(leg_series.ix['2014-01':'2014-06'])